# Chapter 7 합성곱 신경망 (CNN)

## 7.4 합성곱/풀링 계층 구현하기
### 7.4.1 4차원 배열  
데이터의 형상이 (10, 1, 28, 28)이라면, 이는 높이 28, 너비 28, 채널 1개인 데이터가 10개라는 이야기입니다.

In [3]:
import numpy as np

In [4]:
x= np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

In [6]:
x[0].shape

(1, 28, 28)

In [7]:
x[1].shape

(1, 28, 28)

In [8]:
x[0, 0]

array([[6.70568973e-02, 2.66305093e-01, 7.99071716e-01, 3.29396802e-01,
        4.11756377e-01, 4.62223516e-01, 9.49685838e-01, 5.69549736e-01,
        5.12306325e-01, 9.82990762e-01, 2.17953799e-02, 3.08838401e-01,
        3.63372708e-01, 6.33696448e-01, 3.75591310e-01, 6.72967784e-01,
        7.71882532e-01, 8.73981805e-01, 9.37927012e-01, 6.02712214e-01,
        1.80280636e-01, 2.47272900e-02, 2.18514826e-01, 2.83821434e-01,
        4.09381874e-01, 7.82228912e-02, 4.59316395e-01, 5.99748621e-01],
       [2.94223972e-01, 5.21701081e-01, 3.89019785e-01, 3.24964254e-01,
        2.04221913e-01, 6.73959531e-01, 2.61913788e-01, 5.14892513e-01,
        8.53716294e-01, 1.84796115e-02, 9.04964661e-01, 1.85439863e-01,
        7.13160613e-01, 4.45133592e-01, 5.80365798e-01, 8.86890979e-01,
        6.13776567e-01, 6.71367290e-01, 3.11879799e-02, 1.88604051e-02,
        9.49657781e-01, 8.24963256e-01, 1.28035075e-01, 1.62909388e-01,
        4.66836430e-01, 7.91380940e-01, 5.55243654e-02, 6.64976

In [11]:
import sys, os 
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


In [12]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2 * self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2 * self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T # 필터 전개
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

### 7.4.4 풀링 계층 구현하기

풀링 계층 구현도 `im2col`을 사용해 입력 데이트를 전개합니다.  

In [1]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2 * self.pad - self.pool_h) / self.stride)
        out_w = int(1 + (W + 2 * self.pad - self.pool_w) / self.stride)
        
        # 전개 (1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)
        
        # 최댓값 (2)
        out = col.max(axis=1)
        
        # 성형 (3)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out

## 7.5 CNN 구현하기

**초기화때 받는 인수**
|     Parameters     |         내용       |
|--------------------|--------------------|
|input_dim|입력 데이터의 차원|
|conv_param|합성곱 계층의 하이퍼파라미터|
|hidden_size|은닉층의 뉴런 수|
|output_size|출력층의 뉴런 수|
|weight_init_std|초기화 때의 가중치 표준편차|



conv_param 딕셔너리의 키는 다음과 같다.
- filter_num : 필터 수
- filter_size : 필터 크기
- stride : 스트라이드
- pad : 패딩

In [7]:
import numpy as np
from collections import OrderedDict

In [12]:
# 초기화 인수로 주어진 합성곱 계층의 하이퍼파라미터를 딕셔너리에서 꺼내고 합성곱 계층의 출력 크기를 계산한다.
class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28), conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1}, hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size + 2 * filter_pad - filter_size) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size / 2) * (conv_output_size / 2))
        
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size) # Conv layer
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size) # Affine layer (1)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size) # Affine layer (2)
        self.params['b3'] = np.zeros(output_size)
        
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        
        self.last_layer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)
    
    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)
        
        # 역전파
        dout = 1
        dout = self.last_layer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db
        
        return grads

- Relu 함수 구현해야함
- Affine 함수 구현해야함
- SoftmaxWithLoss 함수 구현